## Imports et Chargment des données 

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
match_final_kpi = pd.read_csv(Path("../data/clean/matches_final_kpi.csv"))
dim_teams = pd.read_csv(Path("../data/clean/dim_teams.csv"))
home_stats_normalized = pd.read_csv(Path("../data/clean/home_stats_normalized.csv"))
away_stats_normalized = pd.read_csv(Path("../data/clean/away_stats_normalized.csv"))
matches_normalized = pd.read_csv(Path("../data/clean/matches_normalized.csv"))
teams_reference_normalized = pd.read_csv(Path("../data/clean/teams_reference_normalized.csv"))

print(match_final_kpi.shape, dim_teams.shape, home_stats_normalized.shape, away_stats_normalized.shape, matches_normalized.shape, teams_reference_normalized.shape)

(6814, 11) (228, 6) (6814, 4) (6814, 4) (6814, 7) (228, 2)


# Visualisation de la donnée 

In [3]:
display(dim_teams.head(),
        match_final_kpi.head(),
        home_stats_normalized.head(),
        away_stats_normalized.head(),
        matches_normalized.head(),
        teams_reference_normalized.head())

,team_canonical,team_clean_example,canonical_key,iso2,iso3,team_id
0,Afghanistan,Afghanistan,afghanistan,NaN,NaN,1
1,Albania,Albania,albania,NaN,NaN,2
2,Algeria,Algeria,algeria,NaN,NaN,3
3,American Samoa,American Samoa,americansamoa,NaN,NaN,4
4,Andorra,Andorra,andorra,NaN,NaN,5


,id_match,home_team,away_team,home_result,away_result,result,date,round,city,edition,is_final
0,1,France,Mexico,4,1,France,1930-07-13,Group,Montevideo,1930,True
1,2,Romania,Peru,3,1,Romania,1930-07-14,Group,Montevideo,1930,True
2,3,Yugoslavia,Brazil,2,1,Yugoslavia,1930-07-14,Group,Montevideo,1930,True
3,4,Argentina,France,1,0,Argentina,1930-07-15,Group,Montevideo,1930,True
4,5,Chile,Mexico,3,0,Chile,1930-07-16,Group,Montevideo,1930,True


,id_match,id_team,Number_of_goals_scored,Number_of_goals_conceded
0,1,77,4,1
1,2,164,3,1
2,3,226,2,1
3,4,10,1,0
4,5,41,3,0


,id_match,id_team,Number_of_goals_scored,Number_of_goals_conceded
0,1,131,1,4
1,2,157,1,3
2,3,28,1,2
3,4,77,0,1
4,5,131,0,3


,id_match,result,date,round,city,edition,is_final
0,1,77,1930-07-13,Group,Montevideo,1930,False
1,2,164,1930-07-14,Group,Montevideo,1930,False
2,3,226,1930-07-14,Group,Montevideo,1930,False
3,4,10,1930-07-15,Group,Montevideo,1930,False
4,5,41,1930-07-16,Group,Montevideo,1930,False


,id_team,Team_name
0,1,Afghanistan
1,2,Albania
2,3,Algeria
3,4,American Samoa
4,5,Andorra


# 📊 ANALYSE EXPLORATOIRE & DATA STORY FIFA WORLD CUP
## Histoire des Coupes du Monde à travers les données (1930-2022)

Cette analyse révèle **92 ans d'histoire footballistique** à travers les matchs, buts, et performances des équipes nationales.

In [4]:
# Import des bibliothèques de visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Configuration globale des graphiques
plt.style.use('default')
sns.set_palette("husl")
px.defaults.template = "plotly_white"

print("✅ Bibliothèques de visualisation chargées")

✅ Bibliothèques de visualisation chargées


## 🌍 CHAPITRE 1 : Vue d'ensemble - L'évolution du football mondial
### Combien de matchs, de buts, d'équipes ? Que nous disent les chiffres globaux ?

In [36]:
matches_normalized[(matches_normalized["is_final"]== True)]["round"].value_counts()

round
Quarter-finals    82
Semi-finals       62
Final             44
Name: count, dtype: int64

In [38]:
match_final_kpi[match_final_kpi["edition"]== 2018].shape

(16, 11)

In [37]:
match_final_kpi[match_final_kpi["is_final"]== True].groupby('edition').size().reset_index(name='nb_matches').tail()

,edition,nb_matches
17,2006,74
18,2010,62
19,2014,65
20,2018,16
21,2022,64


In [34]:
# 📈 VISUALISATION 1 : Évolution du nombre de matchs par Coupe du Monde
matches_by_edition = match_final_kpi[match_final_kpi["is_final"]== True].groupby('edition').size().reset_index(name='nb_matches')
matches_by_edition['edition'] = matches_by_edition['edition'].astype(int)

fig = px.bar(matches_by_edition.sort_values('edition'), 
             x='edition', y='nb_matches',
             title="🏆 Évolution du nombre de matchs par Coupe du Monde (1930-2022)",
             labels={'edition': 'Année de la Coupe du Monde', 'nb_matches': 'Nombre de matchs'},
             color='nb_matches',
             color_continuous_scale='viridis')

fig.update_layout(height=500)
fig.show()

print(f"📊 Insight: De {matches_by_edition['nb_matches'].min()} matchs en 1930 à {matches_by_edition['nb_matches'].max()} matchs au maximum")

📊 Insight: De 15 matchs en 1930 à 74 matchs au maximum


In [45]:
# 🎯 VISUALISATION 2 : Total des buts par édition - L'explosion offensive
# Calcul des buts totaux par match puis par édition
goals_by_match = pd.merge(matches_normalized[matches_normalized["is_final"]==True], home_stats_normalized, on='id_match', how='inner')
goals_by_match = pd.merge(goals_by_match, away_stats_normalized, on='id_match', suffixes=('_home', '_away'), how='inner')
goals_by_match['total_goals'] = goals_by_match['Number_of_goals_scored_home'] + goals_by_match['Number_of_goals_scored_away']

goals_by_edition = goals_by_match.groupby('edition').agg({
    'total_goals': 'sum',
    'id_match': 'count'
}).reset_index()
goals_by_edition['avg_goals_per_match'] = goals_by_edition['total_goals'] / goals_by_edition['id_match']
goals_by_edition['edition'] = goals_by_edition['edition'].astype(int)

# Graphique double axe
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x=goals_by_edition['edition'], y=goals_by_edition['total_goals'], 
                     name="Total buts", marker_color='lightblue'), secondary_y=False)

fig.add_trace(go.Scatter(x=goals_by_edition['edition'], y=goals_by_edition['avg_goals_per_match'], 
                         mode='lines+markers', name="Moyenne buts/match", 
                         line=dict(color='red', width=3)), secondary_y=True)

fig.update_xaxes(title_text="Année")
fig.update_yaxes(title_text="Total buts", secondary_y=False)
fig.update_yaxes(title_text="Buts par match", secondary_y=True)
fig.update_layout(title="⚽ Évolution des buts : Total vs Moyenne par match")
fig.show()

print(f"🎯 L'édition la plus prolifique: {goals_by_edition.loc[goals_by_edition['total_goals'].idxmax(), 'edition']} avec {goals_by_edition['total_goals'].max()} buts")

🎯 L'édition la plus prolifique: 1938 avec 78 buts


In [75]:
goals_by_match["total_goals"].sum()

620

In [76]:
# 🇫🇷 ANALYSE SPÉCIALE : Performance de l'équipe de France
print("🇫🇷 ANALYSE ÉQUIPE DE FRANCE - COUPE DU MONDE")
print("=" * 55)

# Filtrer les données pour la France uniquement
france_stats = combined_stats[(combined_stats['Team_name'] == 'France')].copy()

if len(france_stats) > 0:
    # Statistiques générales
    total_matches_france = len(france_stats)
    total_goals_scored = france_stats['Number_of_goals_scored'].sum()
    total_goals_conceded = france_stats['Number_of_goals_conceded'].sum()
    goal_difference = total_goals_scored - total_goals_conceded
    
    # Calcul des victoires, nuls, défaites
    # On récupère les matchs avec résultats détaillés
    france_detailed = []
    for _, match in france_stats.iterrows():
        match_id = match['id_match']
        # Récupérer le score du match
        home_stat = home_stats_normalized[home_stats_normalized['id_match'] == match_id]
        away_stat = away_stats_normalized[away_stats_normalized['id_match'] == match_id]
        
        if not home_stat.empty and not away_stat.empty:
            home_goals = home_stat['Number_of_goals_scored'].iloc[0]
            away_goals = away_stat['Number_of_goals_scored'].iloc[0]
            
            # Déterminer si la France était à domicile ou extérieur
            if match['location'] == 'home':
                france_goals = home_goals
                opponent_goals = away_goals
            else:
                france_goals = away_goals
                opponent_goals = home_goals
            
            if france_goals > opponent_goals:
                result = 'Victoire'
            elif france_goals < opponent_goals:
                result = 'Défaite'
            else:
                result = 'Match Nul'
                
            france_detailed.append({
                'result': result,
                'france_goals': france_goals,
                'opponent_goals': opponent_goals
            })
    
    # Compter les résultats
    france_results = pd.DataFrame(france_detailed)
    if not france_results.empty:
        victories = len(france_results[france_results['result'] == 'Victoire'])
        draws = len(france_results[france_results['result'] == 'Match Nul'])
        defeats = len(france_results[france_results['result'] == 'Défaite'])
        
        # Pourcentage de réussite (victoires + nuls)
        success_rate = ((victories + draws) / total_matches_france) * 100
        victory_rate = (victories / total_matches_france) * 100
    else:
        victories = draws = defeats = 0
        success_rate = victory_rate = 0
    
    print(f"📊 STATISTIQUES GÉNÉRALES")
    print(f"   • Nombre total de matchs joués : {total_matches_france}")
    print(f"   • Participations en Coupe du Monde analysées")
    
    print(f"\n🏆 BILAN SPORTIF")
    print(f"   • Victoires : {victories} ({victory_rate:.1f}%)")
    print(f"   • Matchs nuls : {draws} ({(draws/total_matches_france)*100 if total_matches_france > 0 else 0:.1f}%)")
    print(f"   • Défaites : {defeats} ({(defeats/total_matches_france)*100 if total_matches_france > 0 else 0:.1f}%)")
    
    print(f"\n⚽ STATISTIQUES OFFENSIVES/DÉFENSIVES")
    print(f"   • Buts marqués : {total_goals_scored}")
    print(f"   • Buts encaissés : {total_goals_conceded}")
    print(f"   • Différence de buts : {goal_difference:+d}")
    print(f"   • Moyenne buts marqués/match : {total_goals_scored/total_matches_france:.2f}")
    print(f"   • Moyenne buts encaissés/match : {total_goals_conceded/total_matches_france:.2f}")
    
    print(f"\n🎯 TAUX DE RÉUSSITE")
    print(f"   • Pourcentage de victoires : {victory_rate:.1f}%")
    print(f"   • Pourcentage de réussite (V+N) : {success_rate:.1f}%")
    
    # Conclusion dynamique
    print(f"\n💡 CONCLUSION")
    if goal_difference > 0:
        offensive_performance = "solide attaque"
    elif goal_difference == 0:
        offensive_performance = "équilibre offensif-défensif"
    else:
        offensive_performance = "défense à améliorer"
    
    if victory_rate > 50:
        overall_performance = "excellente performance"
    elif victory_rate > 33:
        overall_performance = "performance honorable"
    else:
        overall_performance = "marge de progression importante"
    
    print(f"   🇫🇷 La France affiche une {overall_performance} en Coupe du Monde")
    print(f"   📈 Avec {total_matches_france} matchs et une {offensive_performance}")
    print(f"   🏆 Taux de réussite de {success_rate:.1f}% (victoires + nuls)")
    
    if goal_difference > 10:
        print(f"   ⭐ Différentiel de +{goal_difference} buts témoigne d'une grande efficacité")
    elif goal_difference > 0:
        print(f"   ✅ Différentiel positif de +{goal_difference} buts, signe d'une équipe compétitive")
    elif goal_difference == 0:
        print(f"   ⚖️ Équilibre parfait entre attaque et défense")
    else:
        print(f"   ⚠️ Différentiel négatif de {goal_difference} buts, axes d'amélioration identifiés")

else:
    print("❌ Aucune donnée trouvée pour l'équipe de France")
    print("   Vérifier l'orthographe : 'France' dans les données")

🇫🇷 ANALYSE ÉQUIPE DE FRANCE - COUPE DU MONDE
📊 STATISTIQUES GÉNÉRALES
   • Nombre total de matchs joués : 8
   • Participations en Coupe du Monde analysées

🏆 BILAN SPORTIF
   • Victoires : 7 (87.5%)
   • Matchs nuls : 0 (0.0%)
   • Défaites : 1 (12.5%)

⚽ STATISTIQUES OFFENSIVES/DÉFENSIVES
   • Buts marqués : 20
   • Buts encaissés : 6
   • Différence de buts : +14
   • Moyenne buts marqués/match : 2.50
   • Moyenne buts encaissés/match : 0.75

🎯 TAUX DE RÉUSSITE
   • Pourcentage de victoires : 87.5%
   • Pourcentage de réussite (V+N) : 87.5%

💡 CONCLUSION
   🇫🇷 La France affiche une excellente performance en Coupe du Monde
   📈 Avec 8 matchs et une solide attaque
   🏆 Taux de réussite de 87.5% (victoires + nuls)
   ⭐ Différentiel de +14 buts témoigne d'une grande efficacité


## 🏟️ CHAPITRE 2 : Avantage du terrain - Home vs Away
### Le mythe de l'avantage à domicile résiste-t-il à l'analyse des Coupes du Monde ?

In [ ]:
matches_normalized

In [ ]:
# 🏠 VISUALISATION 3 : Performance Home vs Away - Le verdict des données
# Comparaison des performances domicile vs extérieur
home_away_comparison = pd.merge(home_stats_normalized, away_stats_normalized, on='id_match', suffixes=('_home', '_away'))

# Classification des résultats
def classify_result(row):
    if row['number_of_goals_scored_home'] > row['number_of_goals_scored_away']:
        return 'Victoire Domicile'
    elif row['number_of_goals_scored_home'] < row['number_of_goals_scored_away']:
        return 'Victoire Extérieur'
    else:
        return 'Match Nul'

home_away_comparison['result_type'] = home_away_comparison.apply(classify_result, axis=1)
result_counts = home_away_comparison['result_type'].value_counts()

# Graphique en camembert avec style moderne
fig = px.pie(values=result_counts.values, names=result_counts.index,
             title="🏟️ Répartition des résultats : Domicile vs Extérieur vs Nul",
             color_discrete_sequence=['#FF6B6B', '#4ECDC4', '#45B7D1'])

fig.update_traces(textposition='inside', textinfo='percent+label', textfont_size=12)
fig.update_layout(height=500)
fig.show()

print("📊 Statistiques détaillées:")
for result_type, count in result_counts.items():
    percentage = (count / len(home_away_comparison)) * 100
    print(f"   {result_type}: {count} matchs ({percentage:.1f}%)")

## 👑 CHAPITRE 3 : Les légendes du football - Top équipes
### Qui sont les vrais maîtres des Coupes du Monde ? Analyse des performances par équipe

In [50]:
matches_normalized.columns

Index(['id_match', 'result', 'date', 'round', 'city', 'edition', 'is_final'], dtype='object')

In [70]:
# 👑 VISUALISATION 4 : TOP 15 des équipes les plus actives
# Calcul des statistiques par équipe
all_team_stats = []

# Statistiques domicile
home_team_stats = home_stats_normalized.merge(teams_reference_normalized, 
                                            left_on='id_team', right_on='id_team')
home_team_stats['location'] = 'home'

# Statistiques extérieur  
away_team_stats = away_stats_normalized.merge(teams_reference_normalized,
                                            left_on='id_team', right_on='id_team')
away_team_stats['location'] = 'away'

# Combinaison des statistiques
combined_stats = pd.concat([
    home_team_stats[['Team_name', 'Number_of_goals_scored', 'Number_of_goals_conceded', 'location','id_match']],
    away_team_stats[['Team_name', 'Number_of_goals_scored', 'Number_of_goals_conceded', 'location']]
])
combined_stats = pd.merge(
    combined_stats,
    matches_normalized[matches_normalized["is_final"]==True], 
    on='id_match',
    how='inner'
)
team_summary = combined_stats.groupby('Team_name').agg({
    'Number_of_goals_scored': 'sum',
    'Number_of_goals_conceded': 'sum',
    'Team_name': 'count'  # Nombre de matchs
}).rename(columns={'Team_name': 'total_matches'})

team_summary['goal_difference'] = team_summary['Number_of_goals_scored'] - team_summary['Number_of_goals_conceded']
team_summary = team_summary.sort_values('total_matches', ascending=False).head(15)

# Graphique horizontal moderne
fig = px.bar(team_summary.reset_index().head(15), 
             x='total_matches', y='Team_name', 
             orientation='h',
             title="🏆 Top 15 des équipes les plus actives en Coupe du Monde",
             labels={'total_matches': 'Nombre de matchs joués', 'Team_name': 'Équipe'},
             color='goal_difference',
             color_continuous_scale='RdYlGn',
             text='total_matches')

fig.update_traces(texttemplate='%{text}', textposition='inside')
fig.update_layout(height=600, yaxis={'categoryorder':'total ascending'})
fig.show()

print("🎯 Top 3 des équipes les plus actives:")
for i, (team, data) in enumerate(team_summary.head(3).iterrows()):
    print(f"   {i+1}. {team}: {data['total_matches']} matchs, {data['Number_of_goals_scored']} buts marqués")

🎯 Top 3 des équipes les plus actives:
   1. Brazil: 25 matchs, 70 buts marqués
   2. Italy: 18 matchs, 34 buts marqués
   3. Netherlands: 15 matchs, 33 buts marqués


In [71]:
# ⚔️ VISUALISATION 5 : Scatter Plot - Efficacité offensive vs défensive
# Analyse de l'efficacité des équipes (top 20)
top_teams = team_summary.head(20).reset_index()

fig = px.scatter(top_teams, 
                x='Number_of_goals_scored', 
                y='Number_of_goals_conceded',
                size='total_matches',
                color='goal_difference',
                hover_name='Team_name',
                title="⚔️ Efficacité des équipes : Attaque vs Défense",
                labels={
                    'Number_of_goals_scored': 'Buts marqués', 
                    'Number_of_goals_conceded': 'Buts encaissés',
                    'goal_difference': 'Différence de buts'
                },
                color_continuous_scale='RdYlGn',
                size_max=60)

# Ligne de référence (performance équilibrée)
fig.add_shape(type="line", x0=0, y0=0, 
              x1=top_teams['Number_of_goals_scored'].max(), 
              y1=top_teams['Number_of_goals_scored'].max(),
              line=dict(color="gray", dash="dash", width=2))

fig.update_layout(height=600)
fig.show()

print("💡 Lecture du graphique:")
print("   🟢 Équipes en vert (au-dessus de la ligne): Plus de buts marqués qu'encaissés")
print("   🔴 Équipes en rouge (en-dessous de la ligne): Plus de buts encaissés que marqués")
print("   📊 Taille des bulles = Nombre de matchs joués")

💡 Lecture du graphique:
   🟢 Équipes en vert (au-dessus de la ligne): Plus de buts marqués qu'encaissés
   🔴 Équipes en rouge (en-dessous de la ligne): Plus de buts encaissés que marqués
   📊 Taille des bulles = Nombre de matchs joués


## 🎭 CHAPITRE 4 : Data Story - Les révélations cachées
### Synthèse narrative : Que nous révèlent vraiment 92 ans de Coupes du Monde ?

In [69]:
# 📊 DATA STORY - Synthèse des insights clés
print("🏆 DATA STORY FIFA WORLD CUP (1930-2022)")
print("=" * 50)

# Calculs pour la story
total_matches = len(matches_normalized[matches_normalized["is_final"]==True])
total_teams = len(teams_reference_normalized)
total_goals = combined_stats['Number_of_goals_scored'].sum()
avg_goals_per_match = total_goals / total_matches

# Most active team
most_active_team = team_summary.index[0]
most_active_matches = team_summary.iloc[0]['total_matches']

# # Home advantage calculation
# home_wins = len(home_away_comparison[home_away_comparison['result_type'] == 'Victoire Domicile'])
# away_wins = len(home_away_comparison[home_away_comparison['result_type'] == 'Victoire Extérieur'])
# draws = len(home_away_comparison[home_away_comparison['result_type'] == 'Match Nul'])

print(f"📈 CROISSANCE DU TOURNOI:")
print(f"   • {total_matches} matchs disputés au total")
print(f"   • {total_teams} équipes différentes ont participé")
# print(f"   • Évolution: de ~15 matchs en 1930 à 64+ matchs aujourd'hui")

print(f"\n⚽ EXPLOSION OFFENSIVE:")
print(f"   • {total_goals} buts marqués au total")
print(f"   • Moyenne: {avg_goals_per_match:.2f} buts par match")
print(f"   • Le football devient de plus en plus spectaculaire!")

# print(f"\n🏟️ MYTHE DE L'AVANTAGE À DOMICILE:")
# print(f"   • Victoires domicile: {home_wins} ({(home_wins/total_matches)*100:.1f}%)")
# print(f"   • Victoires extérieur: {away_wins} ({(away_wins/total_matches)*100:.1f}%)")
# print(f"   • Matchs nuls: {draws} ({(draws/total_matches)*100:.1f}%)")
# if home_wins > away_wins:
#     print("   ✅ L'avantage domicile existe bel et bien en Coupe du Monde!")
# else:
#     print("   ❌ Pas d'avantage domicile significatif en Coupe du Monde!")

print(f"\n👑 HIÉRARCHIE DES NATIONS:")
print(f"   • Équipe la plus active: {most_active_team} ({most_active_matches} matchs)")
print(f"   • Les grandes nations dominent par leur régularité")
print(f"   • La différence de buts révèle les vraies hiérarchies")

🏆 DATA STORY FIFA WORLD CUP (1930-2022)
📈 CROISSANCE DU TOURNOI:
   • 188 matchs disputés au total
   • 228 équipes différentes ont participé

⚽ EXPLOSION OFFENSIVE:
   • 406 buts marqués au total
   • Moyenne: 2.16 buts par match
   • Le football devient de plus en plus spectaculaire!

👑 HIÉRARCHIE DES NATIONS:
   • Équipe la plus active: Brazil (25 matchs)
   • Les grandes nations dominent par leur régularité
   • La différence de buts révèle les vraies hiérarchies


## 🚀 BONUS : Visualisations supplémentaires 
### Explorez d'autres angles d'analyse intéressants

In [ ]:
# 🌡️ VISUALISATION BONUS : Heatmap des performances par décennie
# Analyse temporelle des performances
matches_with_goals = goals_by_match.copy()
matches_with_goals['decade'] = (matches_with_goals['edition'].astype(int) // 10) * 10

decade_stats = matches_with_goals.groupby(['decade']).agg({
    'total_goals': ['mean', 'sum', 'count']
}).round(2)

decade_stats.columns = ['Moyenne_buts_match', 'Total_buts', 'Nombre_matchs']
decade_stats = decade_stats.reset_index()

# Heatmap avec Plotly
fig = px.imshow(decade_stats.set_index('decade').T, 
                aspect="auto",
                title="🌡️ Évolution des performances par décennie",
                labels=dict(x="Décennie", y="Métrique", color="Valeur"),
                color_continuous_scale="viridis")

fig.update_layout(height=400)
fig.show()

print("📊 Analyse par décennie:")
for _, row in decade_stats.iterrows():
    print(f"   {row['decade']}s: {row['Moyenne_buts_match']:.2f} buts/match, {row['Nombre_matchs']} matchs")